In [2]:
import requests
import pandas as pd
!pip install pymysql sqlalchemy

In [3]:
url = "http://rest.nbaapi.com/api/PlayerDataAdvanced/team/LAL"
res = requests.get(url)
data = res.json()

In [4]:
import json
print(json.dumps(data[0], indent=2))  # Should now show one player's season stats

{
  "id": 28,
  "playerName": "Mo Bamba",
  "position": "C",
  "age": 24,
  "games": 9,
  "minutesPlayed": 88,
  "per": 11.4,
  "tsPercent": 0.518,
  "threePAR": 0.593,
  "ftr": 0.407,
  "offensiveRBPercent": 13.9,
  "defensiveRBPercent": 35.3,
  "totalRBPercent": 24.9,
  "assistPercent": 6.0,
  "stealPercent": 0.5,
  "blockPercent": 4.8,
  "turnoverPercent": 20.1,
  "usagePercent": 19.1,
  "offensiveWS": -0.1,
  "defensiveWS": 0.2,
  "winShares": 0.1,
  "winSharesPer": 0.054,
  "offensiveBox": -5.3,
  "defensiveBox": -0.2,
  "box": -5.5,
  "vorp": -0.1,
  "team": "LAL",
  "season": 2023,
  "playerId": "bambamo01"
}


In [5]:
import requests

url = "http://rest.nbaapi.com/api/PlayerDataAdvanced/team/LAL"
res = requests.get(url)
print("Status:", res.status_code)
print("Response:", res.text)


Status: 200
Response: [{"id":28,"playerName":"Mo Bamba","position":"C","age":24,"games":9,"minutesPlayed":88,"per":11.4,"tsPercent":0.518,"threePAR":0.593,"ftr":0.407,"offensiveRBPercent":13.900,"defensiveRBPercent":35.300,"totalRBPercent":24.900,"assistPercent":6.000,"stealPercent":0.500,"blockPercent":4.800,"turnoverPercent":20.100,"usagePercent":19.100,"offensiveWS":-0.1,"defensiveWS":0.2,"winShares":0.1,"winSharesPer":0.054,"offensiveBox":-5.3,"defensiveBox":-0.2,"box":-5.5,"vorp":-0.1,"team":"LAL","season":2023,"playerId":"bambamo01"},{"id":48,"playerName":"Malik Beasley","position":"SG","age":26,"games":26,"minutesPlayed":621,"per":9.7,"tsPercent":0.521,"threePAR":0.698,"ftr":0.078,"offensiveRBPercent":1.400,"defensiveRBPercent":13.000,"totalRBPercent":7.400,"assistPercent":7.000,"stealPercent":1.600,"blockPercent":0.100,"turnoverPercent":9.800,"usagePercent":20.900,"offensiveWS":-0.5,"defensiveWS":0.6,"winShares":0.2,"winSharesPer":0.012,"offensiveBox":-2.3,"defensiveBox":-0.6,"

In [ ]:
teams = ["ATL", "BOS", "BRK", "CHA", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP", "NYK", "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"]

In [7]:
def get_team_data(team_code):
    url = f"http://rest.nbaapi.com/api/PlayerDataAdvanced/team/{team_code}"
    response = requests.get(url)

    # Safety check
    if response.status_code != 200:
        print(f"❌ Failed for team {team_code}: {response.status_code}")
        return pd.DataFrame()  # Return empty if it failed

    try:
        data = response.json()
    except Exception as e:
        print(f"❌ JSON decode error for {team_code}: {e}")
        return pd.DataFrame()

    # ✅ Confirm it's a list
    if not isinstance(data, list):
        print(f"❌ Unexpected format for team {team_code}: not a list")
        print("Raw data:", data)
        return pd.DataFrame()

    # Parse each player
    player_stats = []
    for player in data:
        player_stats.append({
            "player": player["playerName"],
            "team": player["team"],
            "season": player["season"],
            "games": player["games"],
            "minutes_played": player["minutesPlayed"],
            "per": player["per"],
            "ts_percent": player["tsPercent"],
            "usage_percent": player["usagePercent"],
            "win_shares": player["winShares"],
            "win_shares_per48": player["winSharesPer"],
            "vorp": player["vorp"],
            "box_plus_minus": player["box"]
        })

    return pd.DataFrame(player_stats)


In [8]:
top_players_all_seasons = []

seasons = range(2022, 2026)  # 2021 through 2025

for season in seasons:
    for team in teams:
        df_team = get_team_data(team)

        # Filter for the current season
        df_team = df_team[df_team["season"] == season]

        # Skip if no data
        if df_team.empty:
            continue

        # Get top 5 players by win shares
        top_players = df_team.sort_values(by="win_shares", ascending=False).head(5)

        top_players_all_seasons.append(top_players)

# Combine all top players into one DataFrame
df_top_players = pd.concat(top_players_all_seasons, ignore_index=True)

# Preview
print(df_top_players)

# Save to CSV
df_top_players.to_csv("top_players_by_team_2021_2025.csv", index=False)


                player team  season  games  minutes_played   per  ts_percent  \
0           Trae Young  ATL    2022     76            2652  25.4       0.603   
1         Clint Capela  ATL    2022     74            2042  21.4       0.604   
2         John Collins  ATL    2022     54            1663  18.7       0.610   
3       Onyeka Okongwu  ATL    2022     48             992  19.8       0.714   
4    Bogdan Bogdanović  ATL    2022     63            1848  15.4       0.562   
..                 ...  ...     ...    ...             ...   ...         ...   
575  Justin Champagnie  WAS    2025     62            1340  15.7       0.614   
576  Jonas Valančiūnas  WAS    2025     49             983  20.6       0.602   
577     Richaun Holmes  WAS    2025     31             534  19.8       0.695   
578       Jordan Poole  WAS    2025     68            2001  16.6       0.591   
579      Corey Kispert  WAS    2025     61            1602  11.0       0.584   

     usage_percent  win_shares  win_sha

In [10]:
from sqlalchemy import create_engine

In [14]:
host = "isba-dev-02.ccns0o4cgs58.us-east-1.rds.amazonaws.com"
user = "postgres"
password = "isba_4715"
port = 5432
database = "postgres"

In [15]:
# Build the connection string
pg_conn_str = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'

# Create engine
engine = create_engine(pg_conn_str)

# Load your DataFrame to your desired table
df_top_players.to_sql("nba_api_player_stats", engine, if_exists="replace", index=False)

580